In [1]:
import numpy as np
import NUTS as hmc
import matplotlib.pyplot as plt
import corner
import time
import sklearn.datasets as skld

In [2]:
plt.rcParams['figure.figsize'] = [10, 10]

limits = [(-40,40),(-40,40),(-40,40)]
nparam = len(limits)

cov = skld.make_spd_matrix(nparam)
#cov = np.eye(nparam)*100
inv_cov = np.linalg.inv(cov)


In [3]:
def mock_posterior_and_gradient(p):
    dlogL_dCl = - inv_cov @ p
    #j = theory_jacobian(p, *self.args).T
    logP = 0.5 * p.T @ dlogL_dCl
    #logP_jacobian = j @ dlogL_dCl
    #logP_jacobian = p * (-1)
    #mock_posterior_and_gradient.counter += 1
    return logP, dlogL_dCl

In [4]:
def run_hmc(n_it, hmc, cov, burnin):
    #rank = 5
    rank = nparam
    #np.random.seed(100 + rank)
    #C = np.eye(nparam)
    # mass matrix
    sampler = hmc.HMC(mock_posterior_and_gradient, cov, limits)
    # first sample starts at fid
    fid_params  = np.zeros(nparam)
    results = sampler.NUTS(fid_params, 0.6, n_it, burnin)

    return sampler

In [ ]:
start = time.time()
nit = 1000
burnin = 100
chain = run_hmc(nit, hmc, cov, burnin)
print("%.2f" %(time.time()-start))

Reasonable epsilon is 1.5
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 3
Sample accepted j = 5
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 3
Sample accepted j = 4
Sample accepted j = 5
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 3
Sample accepted j = 4
Sample accepted j = 5
Sample accepted j = 6
Sample accepted j = 7
Sample accepted j = 8
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 3
Sample accepted j = 4
Sample accepted j = 5
Sample accepted j = 6
Sample accepted j = 7
Sample accepted j = 8
Sample accepted j = 9
Sample accepted j = 10
Sample accepted j = 0
Sample accepted j = 1
Sample accepted j = 2
Sample accepted j = 3
Sample accepted j = 4
Sample accepted j = 5
Sample accepted j = 6
Sample accepted j = 7
Sample accepted j = 8
Sample accepted j = 9
Sampl

In [ ]:
paths = np.array(chain.paths)
#anti_paths = np.array(chain.anti_paths)
#trace = np.array(chain.trace)

calls = np.array(chain.ncall_list)

In [ ]:
plt.scatter(paths[:,0],paths[:,1],s=3)
#plt.scatter(anti_paths[:,0],anti_paths[:,1],s=3)
plt.show()

#plt.scatter(trace[:,0],trace[:,1],s=5)
#plt.show()

#figure = corner.corner(trace)

In [ ]:
def plot_conv(trace,dim,botlim,toplim, hmc, cov, calls):
    STDs, means = [], []
    for i in range(trace.shape[0]):
        STD=np.std(trace[:i,dim])
        STDs.append(STD/np.sqrt(cov[dim,dim]))
        means.append(np.mean(trace[:i,dim]))
    plt.plot(calls, STDs, label="STD/sigma hmc %s" %hmc)
    plt.plot(calls, means, label="mean hmc %s" %hmc)
    plt.hlines([0,1],0,calls[-1],ls=":", color="k")
    plt.legend()
    plt.title("Mean & STD vs L-calls for dim %.0f" %dim)
    plt.ylim(botlim,toplim)
    
    print(np.std(trace[:,dim]))
    

In [ ]:
print(np.sqrt(cov))

plot_conv(trace,2,-1,1.5,1, cov, calls)
plot_conv(trace2,2,-1,1.5,2, cov, calls2)
plt.show()